In [0]:
CREATE OR REPLACE TEMPORARY FUNCTION clean_date(d DATE)
RETURNS DATE
RETURN CASE
  WHEN d IS NULL THEN NULL
  WHEN d < DATE '1901-01-01' THEN NULL
  WHEN d IN (
    DATE '1900-01-01', DATE '1901-01-01',
    DATE '1969-12-31', DATE '1970-01-01',
    DATE '2099-12-31', DATE '2999-12-31', DATE '9999-12-31'
  ) THEN NULL
  WHEN d > current_date() + INTERVAL 3 YEARS THEN NULL
  ELSE d
END;

--CREATE OR REPLACE TABLE teamconnect.pca_prism.08_tbl_prm_shaped AS
INSERT OVERWRITE TABLE teamconnect.pca_prism.08_tbl_prm_shaped
SELECT *
FROM (
  SELECT
    CASE 
      WHEN LENGTH(exc_digits) = 6 AND LENGTH(lne_digits) BETWEEN 1 AND 4
        THEN CONCAT(exc_digits, LPAD(lne_digits, 4, '0'))
      ELSE NULL
    END AS btn,
    dpi_environment,
    promo_code,
    promo_start,
    promo_end,
    promo_amount
  FROM (
    SELECT
      REGEXP_REPLACE(CAST(psexc AS STRING),'[^0-9]','') AS exc_digits,
      REGEXP_REPLACE(CAST(pslin AS STRING),'[^0-9]','') AS lne_digits,
      UPPER(TRIM(env))  AS dpi_environment,
      UPPER(TRIM(pscod)) AS promo_code,
      clean_date(TRY_TO_DATE(CAST(pscsdt AS STRING),'yyyyMMdd')) AS promo_start,
      clean_date(TRY_TO_DATE(CAST(pspedt AS STRING),'yyyyMMdd')) AS promo_end,
      CAST(psmsa AS DECIMAL(18,2)) AS promo_amount
    FROM it_bronze_prod.dpi.subprm
  ) r
) t
WHERE promo_code IS NOT NULL
  AND btn IS NOT NULL
  AND btn RLIKE '^[0-9]{10}$';

OPTIMIZE teamconnect.pca_prism.08_tbl_prm_shaped
ZORDER BY (btn, dpi_environment);
